In [260]:
import pandas as pd
import numpy as np
import re
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import string
import joblib
import function_library as fl
import math
import plotly.express as px
import pickle
import dash
import plotly.express as px
import json
import h5py
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras.layers import Embedding
from gensim.models import Word2Vec
from dash import dcc, html, Input, Output
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from spacy.lang.pt.stop_words import STOP_WORDS
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [261]:
pd.set_option('display.max_colwidth', 1000)

In [262]:
all_cus_go = pd.read_csv('F:\\project_csv\\ALL_PL_2022.csv', sep=';')

In [263]:
email_encoder = LabelEncoder()

all_cus_go['Requester Email'] = email_encoder.fit_transform(all_cus_go['Requester Email'])

In [264]:
all_cus_go.head()

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
0,111345,[DE] [PYTANIE] B07R42W8J9,Klient ma pytanie do produktu B07R42W8J9: &quot;Jakie połączenie ma dachówka wentylacyjna?&quot; Link do zapytania: www.amazon.de/ask/question/Tx21NZNJ8PMI5JH?_encoding=UTF8&amp;authToken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beata,Outbound Email,420,10 01 2022 10:47,4,4,"Partner_wysłane,rynek_de"
1,128291,[DE] 302-0014409-7188317 NIEKOMPLETNE,"Klient zgłasza, że w zamówieniu nie było lampki. Pozdrawiam, Jakub",Outbound Email,235,30 01 2022 09:48,3,2,"Partner_wysłane,rynek_de"
2,128325,[REKLAMACJA] 205-9594094-3558723 UK,"Dzień dobry, Klient napisał: Ten produkt dotarł, ale nie działa, nie świeci się. Bardzo rozczarowany, ponieważ kupiłem go dla mojego dziecka. Brak zauważalnych wad, wszystko wydawało się w porządku po otwarciu. Mały przycisk przesuwa się w dół i wydaje się OK, ale nic się nie dzieje, gdy go nacisnąć. To nie świeci się, co było cały punkt z nim, i bardzo rozczarowujące dla mojej córki. Pozdrawiam, Hania",Outbound Email,395,30 01 2022 10:38,2,2,"Partner_wysłane,rynek_uk,Przypomnienie"
3,131092,[DE] [ZAPYTANIE] 305-6969626-5165121,"Klient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. Pozdrawiam Beata",Outbound Email,279,2 02 2022 16:10,4,4,"Partner_wysłane,rynek_de,Przypomnienie"
4,133983,[IT] 408-8392161-3066735 ZWROT,"https://www.dhl.com/it-en/home/tracking/tracking-parcel.html?submit=1&amp;tracking-id=CM332431606DE Paczka wraca do was, prosimy o refund dla klienta jak dojdzie Kasia",Outbound Email,279,7 02 2022 08:58,9,6,"Partner_wysłane,rynek_it,Przypomnienie1,Przypomnienie2"


1. Data cleaning, Feature engineering

In [265]:
format_date = "%d %m %Y %H:%M"

all_cus_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_cus_go['Created Time']] 
all_cus_go['Subject'] = [str(row).lower() for row in all_cus_go['Subject']]
all_cus_go['Tags'] = [fl.split_string_by_char(row, ',')for row in all_cus_go['Tags']]
all_cus_go['rynek'] = [fl.extract_string_containing_word(row, 'rynek') for row in all_cus_go['Tags']]
all_cus_go['Order_number'] = [fl.extract_order_number(row1, row2, r"\d{3}-\d{7}-\d{7}") for row1, row2 in zip(all_cus_go['Subject'], all_cus_go['Description'])]
all_cus_go['title_list'] = [fl.split_string_by_char(row, ' ') for row in all_cus_go['Subject']]
all_cus_go['label'] = [fl.find_words_with_more_than_four_characters(row) for row in all_cus_go['Subject']]

In [266]:
all_cus_go.head()

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags,rynek,Order_number,title_list,label
0,111345,[de] [pytanie] b07r42w8j9,Klient ma pytanie do produktu B07R42W8J9: &quot;Jakie połączenie ma dachówka wentylacyjna?&quot; Link do zapytania: www.amazon.de/ask/question/Tx21NZNJ8PMI5JH?_encoding=UTF8&amp;authToken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beata,Outbound Email,420,2022-01-10 10:47:00,4,4,"[Partner_wysłane, rynek_de]",rynek_de,,"[[de], [pytanie], b07r42w8j9]",[pytanie]
1,128291,[de] 302-0014409-7188317 niekompletne,"Klient zgłasza, że w zamówieniu nie było lampki. Pozdrawiam, Jakub",Outbound Email,235,2022-01-30 09:48:00,3,2,"[Partner_wysłane, rynek_de]",rynek_de,['302-0014409-7188317'],"[[de], 302-0014409-7188317, niekompletne]",[niekompletne]
2,128325,[reklamacja] 205-9594094-3558723 uk,"Dzień dobry, Klient napisał: Ten produkt dotarł, ale nie działa, nie świeci się. Bardzo rozczarowany, ponieważ kupiłem go dla mojego dziecka. Brak zauważalnych wad, wszystko wydawało się w porządku po otwarciu. Mały przycisk przesuwa się w dół i wydaje się OK, ale nic się nie dzieje, gdy go nacisnąć. To nie świeci się, co było cały punkt z nim, i bardzo rozczarowujące dla mojej córki. Pozdrawiam, Hania",Outbound Email,395,2022-01-30 10:38:00,2,2,"[Partner_wysłane, rynek_uk, Przypomnienie]",rynek_uk,['205-9594094-3558723'],"[[reklamacja], 205-9594094-3558723, uk]",[reklamacja]
3,131092,[de] [zapytanie] 305-6969626-5165121,"Klient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. Pozdrawiam Beata",Outbound Email,279,2022-02-02 16:10:00,4,4,"[Partner_wysłane, rynek_de, Przypomnienie]",rynek_de,['305-6969626-5165121'],"[[de], [zapytanie], 305-6969626-5165121]",[zapytanie]
4,133983,[it] 408-8392161-3066735 zwrot,"https://www.dhl.com/it-en/home/tracking/tracking-parcel.html?submit=1&amp;tracking-id=CM332431606DE Paczka wraca do was, prosimy o refund dla klienta jak dojdzie Kasia",Outbound Email,279,2022-02-07 08:58:00,9,6,"[Partner_wysłane, rynek_it, Przypomnienie1, Przypomnienie2]",rynek_it,['408-8392161-3066735'],"[[it], 408-8392161-3066735, zwrot]",[zwrot]


In [267]:
rynek_encoder = LabelEncoder()

all_cus_go['rynek'] = all_cus_go['rynek'].str.replace('rynek_', '').str.replace('rynek ', '').str.replace('usa','us')
all_cus_go['rynek'] = rynek_encoder.fit_transform(all_cus_go['rynek']) + 1

In [268]:
unique_values = all_cus_go['label'].explode().tolist()
unique_values = [item for item in unique_values if not (isinstance(item, float) and math.isnan(item))]
uniq_vales, uniq_values_count = fl.list_of_unique_values_and_count_of_values(unique_values)
uniq_vales = [str(item) for item in uniq_vales]

In [269]:
nlp = spacy.load('pl_core_news_lg')

uniq_vales_sentence = ' '.join(uniq_vales)

doc = nlp(uniq_vales_sentence)

all_lem = []
all_words = []

for token in doc:
    all_words.append((token.text, token.lemma_))
    all_lem.append(token.lemma_)

dict_of_all_lemmas = {}
for lemma in set(all_lem):
    dict_of_all_lemmas[lemma] = fl.find_all_words_matching_lemma(lemma,all_words)

number_of_occurence = {}
for lemma_key in dict_of_all_lemmas:
    number_of_occurence_value = []
    for word in dict_of_all_lemmas[lemma_key]:
        number_of_occurence_value.append(uniq_values_count[word])
    number_of_occurence[lemma_key] = sum(number_of_occurence_value)

number_of_occurence_sorted = sorted(number_of_occurence.items(), key=lambda x:x[1], reverse=True)

In [270]:
label_regex ={"anulacja" : r'^[a-z]?anu?a?l|^[a-z]?nieanu?a?l',
            "reklamacja" :r'^rek|\w*k{1,}l{1,}a{1,}m{1,}a{1,}\w*',
            "wysyłka" : r'^[a-z]?wysy[łlk]|^[a-z]?niewysy?[łlk]',
            "refund" : r'^[a-z]?r[a-z]?efu',
            "przesyłka" : r'\w*esy[łl]\w*',
            "dostawa" : r'\w*dost[arw]',
            "opóźnienie" : r'^op[oó][zźżx]',
            "uszkodzony" : r'^uszko|\w*kodz\w*',
            "adres" : r'^[a-z]?ad{,2}res|^zaadre|^[a-z]?adr{,2}es',
            "ponowny" : r'^[a-z]?po{0,2}[a-z]?no',
            "paczka" : r'^paczk',
            "pytanie" : r'\w*yta\w*|^zap[yt]',
            "zwrot" : r'^zwr|\w*wrot\w*',
            "brak" : r'^[a-z]?brak',
            "niekompletny" : r'^niekom[ple]{1,}[ple]{1,}|\W*komopl',
            "faktura" : r'^fak',
            "inne" : r'^in[nm]|^i[nm]n'
}


In [271]:
dict_of_labels_and_matching_lemma_keys = {}

for label in label_regex:
    dict_of_labels_and_matching_lemma_keys[label] = []
    pattern = re.compile(label_regex[label])
    for label_key in dict_of_all_lemmas:
        if re.search(pattern, label_key):
            dict_of_labels_and_matching_lemma_keys[label].append(label_key)

not_working_keys, final_dict = fl.match_words_to_labels(dict_of_labels_and_matching_lemma_keys, dict_of_all_lemmas)
len(not_working_keys)

0

In [272]:
all_cus_go['final_label'] = [fl.find_label_in_list_of_words_from_title(final_dict, row) for row in all_cus_go['label']]
labels_distribution = pd.DataFrame(all_cus_go['final_label'].value_counts())
total_count = sum(labels_distribution['final_label'])
labels_distribution['final_label_%'] = [round(count/total_count*100,2) for count in labels_distribution['final_label']]
labels_distribution['final_label_%_cumulative'] = labels_distribution['final_label_%'].cumsum()
labels_distribution = labels_distribution.reset_index()
labels_distribution = labels_distribution.rename(columns={'final_label': 'label_count', 'index' : 'labels'})
labels_distribution.reset_index(drop=True, inplace=True)
labels_distribution

,labels,label_count,final_label_%,final_label_%_cumulative
0,dostawa,8193,22.60,22.60
1,zwrot,5878,16.21,38.81
2,anulacja,5421,14.95,53.76
3,reklamacja,4713,13.00,66.76
4,pytanie,2657,7.33,74.09
5,faktura,1859,5.13,79.22
6,wysyłka,1306,3.60,82.82
7,paczka,1255,3.46,86.28
8,adres,963,2.66,88.94
9,uszkodzony,961,2.65,91.59


2. Exploratory data analysis 

In [273]:
sunburst_df = pd.DataFrame(columns=['ids','label','parent','val', 'val_perc'])

root = {'ids' : 'labels_distribution','label' : 'labels_distribution','parent' : '','val' : all_cus_go['final_label'].count(), 'val_perc' : 100}
sunburst_df = sunburst_df.append(root, ignore_index=True) 

grouped_dict = labels_distribution[['labels', 'label_count']]
grouped_dict.set_index('labels', inplace=True)
grouped_dict = grouped_dict['label_count'].to_dict()

grouped = pd.DataFrame(all_cus_go['final_label'].value_counts())
grouped = grouped.reset_index()
grouped['ids'] = grouped['index']
grouped = grouped.rename(columns={'index' : "label", 'final_label' : 'val'})
grouped['parent'] = "labels_distribution"
grouped['val_perc'] = [fl.count_percentage_value_from_group(grouped_dict, [row1, row2]) for row1, row2 in zip(grouped['label'], grouped['val'])]
sunburst_df = pd.concat([sunburst_df, grouped], ignore_index=True)

grouped = all_cus_go.groupby(['final_label', 'rynek']).size().reset_index(name='val')
grouped['ids'] = [f"{final_label} - {rynek}" for rynek, final_label in zip(grouped["rynek"], grouped["final_label"])]
grouped = grouped.rename(columns={'rynek' : "label", 'final_label' : 'parent'})
grouped['val_perc'] = [fl.count_percentage_value_from_group(grouped_dict, [row1, row2]) for row1, row2 in zip(grouped['parent'], grouped['val'])]
sunburst_df = pd.concat([sunburst_df, grouped])

In [274]:
fig =go.Figure(go.Sunburst(
    ids = sunburst_df.ids,
    labels = sunburst_df.label,
    parents = sunburst_df.parent,
    values = sunburst_df.val_perc,
    branchvalues="total",
    insidetextorientation="radial",
))

fig.update_layout(
    title=dict(text="Labels distribution by country number", font=dict(size=30), automargin=True, yref='paper'),
    margin = dict(t=10, l=10, r=10, b=10),
    autosize=True,
    width=1000,
    height=1000,
)

fig.show()

On the label distribution plot, it is evident that over 50% of messages are distributed among three labels, and over 75% of messages are distributed among six labels. The majority of messages are directed to countries numbered 5, 7, and 8. Utilizing a model that covers these six major labels will likely result in the labeling of most incoming messages. It is recommended for future analysis to determine if any of major label messeges can be answered or partially addressed without the intervention of employees.

In [275]:
time_analys = pd.DataFrame(all_cus_go['Created Time'])
time_analys['day_of_month'] = time_analys['Created Time'].dt.day
time_analys['hours'] = time_analys['Created Time'].dt.hour
time_analys['working_hours'] = ["Working hours" if 8 <= row < 16 else "Not working hours" for row in time_analys['hours']]

df_time_analys = time_analys.groupby(['day_of_month', 'working_hours']).size().reset_index(name='val')
df_time_analys.sort_values(['day_of_month', 'working_hours'],  inplace=True)

time_analys_grouped = df_time_analys.groupby('day_of_month').sum()
time_analys_dict = time_analys_grouped['val'].to_dict()

df_time_analys['perc_val'] = [round(fl.count_percentage_value_for_label(time_analys_dict, [row1, row2]),2) for row1, row2 in zip(df_time_analys['day_of_month'], df_time_analys['val'])]

In [276]:
fig = px.bar(df_time_analys, x='day_of_month', y="perc_val", 
             color='working_hours', text="working_hours", text_auto='.3s',
             labels={'day_of_month':'Day of month', 'perc_val': '% of messages per day', "working_hours": "Hours"},
             title="Messages by day of month and working hours")

fig.show()

In [277]:
time_analys = pd.DataFrame(all_cus_go['Created Time'])
time_analys['day_of_week'] = time_analys['Created Time'].dt.dayofweek
time_analys['day_name'] = time_analys['Created Time'].dt.strftime('%A')
time_analys['hours'] = time_analys['Created Time'].dt.hour
time_analys['hours_group'] = pd.cut(time_analys['hours'], bins=3, labels=['0-7','8-15','16-24'])

df_time_analys = time_analys.groupby(['day_name', 'day_of_week', 'hours_group']).size().reset_index(name='val')
df_time_analys.sort_values(['day_of_week', 'hours_group'],  inplace=True)
df_time_analys = df_time_analys[df_time_analys['val'] != 0]

In [278]:
fig = px.bar(df_time_analys, x="day_name", y="val", color="hours_group", text="hours_group",text_auto=True,
             labels={'day_name':'Day', 'val': 'Number of messages per day', "hours_group": "Hours"},
             title="Messages by day of week and hours")

fig.show()

The plotted distribution of labels by days reveals that approximately 25% to 33% of incoming messages occur outside regular working hours. Remarkably, the message influx remains consistent, demonstrating minimal dependency on the day of the week. However, during weekends, there is an uptick to nearly 50% of the usual volume observed on regular working days. It's recommended to conduct further analysis specifically on weekend messages, focusing on identifying fields that could benefit from automation.

In [279]:
interactions = pd.DataFrame(all_cus_go[['Agent interactions', 'final_label']])

high_volume = ['dostawa', 'zwrot', 'anulacja', 'reklamacja']
medium_volume = ['pytanie', 'faktura', 'wysyłka', 'paczka']

interactions['volume'] = ['High volume' if row in high_volume else 'Medium volume' if row in medium_volume else 'Low volume' for row in interactions['final_label']]

upper_bound = interactions['Agent interactions'].quantile(0.975)

interactions_filtered = interactions[
    (interactions['Agent interactions'] <= upper_bound)
]

fig = px.box(interactions_filtered, x='final_label', y='Agent interactions',
    points='suspectedoutliers', color='volume',
    labels={'final_label':'Label', 'volume' : 'Messages Volume'},
    title="Agents interactions by labels and messages volume"
)
fig.show()

Upon reviewing agent interactions plots, it's evident that nine out of seventeen labels exhibit a count ranging from one to two interactions. The organization should prioritize investigating the 'reklamacja' and 'zwrot' message groups within the high-volume category to understand the reason behind the heightened interaction volume. Simultaneously, it's crucial to explore avenues to minimize the number of interactions within these groups.

Moreover, the 'Dostawa' and 'anulacja' groups merit thorough examination for potential automation possibilities. Should there be success in automating either the 'Dostawa' or 'anulacja' groups, a similar strategy should be implemented for the medium-volume category, where each label experiences interaction counts ranging between one to two.

Description preprocess for modeling 

In [280]:
shortcuts = []
shortcuts_dict = {}
shortcuts_pattern_dict = []

file_path = 'C:\\Users\\48575\\email_classification\\shortcuts.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        shortcuts.append(line.replace('\xa0','').replace('\n','').split(sep='-'))

for shortcut in shortcuts:
    shortcuts_dict[shortcut[0]] = shortcut[1].strip()

for key in shortcuts_dict:
    shortcuts_pattern_dict.append(f'{{"label": "{shortcuts_dict[key]}", "pattern": [{{"TEXT": "{key}"}}]}}')

shortcuts_pattern_dict = [json.loads(item) for item in shortcuts_pattern_dict]

In [281]:
polish_names = []

file_path = 'C:\\Users\\48575\\email_classification\\imiona_polskie.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        polish_names.append({"label" : "persName", "pattern" : [{"TEXT": line.lower().strip()}]})

In [282]:
patterns = [
    {"label" : "number", "pattern" : [{"LIKE_NUM" : True}]},
    {"label" : "web_site", "pattern" : [{"LIKE_URL" : True}]},
    {"label" : "email", "pattern" : [{"LIKE_EMAIL" : True}]},
    {"label" : "ASIN", "pattern" : [{"TEXT" : {"REGEX" : "(?i)b0[a-z0-9]{8}$"}}]}
]
patterns.extend(polish_names)

ruler = nlp.add_pipe("entity_ruler", before='ner')
ruler.add_patterns(patterns)

In [283]:
def text_preprocessing(text: str) -> str:
    """Preprocesses text by deciphering abbreviations and converting words into entities or lemmas.

    This function takes a string as an argument, changes it to an NLP object, iterates through every word, 
    and checks if it's in the shortcuts dictionary. If it finds a word in the dictionary, it retrieves its replacement value,
    processes it further by converting it to an NLP object, and iterates through resulting tokens. For each token, 
    it checks for entity types or lemmas and appends them to the processed tokens list. 
    If a word is not in the shortcuts dictionary, the function checks if it's a number, an entity type, or a lemma. 
    If it's a lemma or an entity type, it's appended to the processed tokens list. The final processed text is joined 
    back into one string.

    Args:
        text (str): String to be processed.

    Returns:
        str: Processed text with lemmas and entities.
    """
    try:
        text = text.lower().replace("&quot;", ' " ')
        processed_tokens = []

        doc = nlp(text)

        for token in doc:  
            if token.text.lower() in shortcuts_dict:
                doc2 = nlp(shortcuts_dict[token.text.lower()])
                for token2 in doc2:
                    if not token.is_stop:
                        if token2.ent_type_ != '':
                            processed_tokens.append(token2.ent_type_)
                        elif not token2.is_punct:
                            processed_tokens.append(token2.lemma_)
            elif token.is_digit:
                processed_tokens.append(token.ent_type_)
            elif not token.is_stop:
                if token.ent_type_ != '':
                    processed_tokens.append(token.ent_type_)
                elif not token.is_punct:
                    processed_tokens.append(token.lemma_)

        processed_text = " ".join(processed_tokens)

        return processed_text
    
    except:
        return text

In [284]:
# df_to_model = all_cus_go[['Subject', 'Description', 'final_label']]
# df_to_model = df_to_model[df_to_model['final_label'].notnull() & df_to_model['Description'].notnull()]
# df_to_model['desc_transformed'] = df_to_model['Description'].apply(text_preprocessing)
# df_to_model.dropna(inplace=True)
# df_to_model.to_csv('F:\\project_csv\\df_to_model.csv')

df_to_model = pd.read_csv('F:\\project_csv\\df_to_model.csv')

In [285]:
df_to_model.dropna(inplace=True)

In [286]:
top_4_labels_and_other = labels_distribution['labels'].head(4)
top_4_labels_and_other = top_4_labels_and_other.to_list()
top_4_labels_and_other

df_to_model['final_label_grouped'] = ['other' if row not in top_4_labels_and_other else row for row in df_to_model['final_label']]
df_to_model['final_label_grouped'].value_counts()

other         12054
dostawa        8193
zwrot          5877
anulacja       5414
reklamacja     4712
Name: final_label_grouped, dtype: int64

tfidf

In [287]:
X = df_to_model['desc_transformed']
y = df_to_model['final_label_grouped']

In [288]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y_train)

In [289]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [290]:
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [291]:
X_train_tf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [292]:
tfidf.get_feature_names_out()

array(['00', '0000', '00340434390970043528', ..., 'żądany', 'żądać', 'żę'],
      dtype=object)

In [293]:
pd.DataFrame(X_train_tf.todense(),
             columns=tfidf.get_feature_names_out())

,00,0000,00340434390970043528,00340434390999113059,00340434391112484421,00340434410325615160,00340434660814398115,0037a7646e21,0042737,0054,...,żywo,żywopłot,żywotność,żółto,żółty,żółwi,żądanie,żądany,żądać,żę
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [294]:
df_functions_score = pd.DataFrame(columns=['model_name', 'label', 'score', 'metric'])


In [295]:
len(y_train)

23200

In [296]:
# lr = LogisticRegression()
# lr.fit(X_train_tf, y_train)

# joblib.dump(lr, "F:\\Classification_models\\LogisticRegression_basic")

In [297]:
lr_loaded = joblib.load("F:\\Classification_models\\LogisticRegression_basic")


y_pred = lr_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "Logistic_regression_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.95      0.97      1083
     dostawa       0.77      0.87      0.82      1639
       other       0.79      0.78      0.78      2411
  reklamacja       0.78      0.75      0.76       942
       zwrot       0.86      0.79      0.83      1175

    accuracy                           0.82      7250
   macro avg       0.84      0.83      0.83      7250
weighted avg       0.82      0.82      0.82      7250



In [298]:
rfc = RandomForestClassifier()
rfc.fit(X_train_tf, y_train)

joblib.dump(rfc, "F:\\Classification_models\\RandomForestClassifier_basic")

['F:\\Classification_models\\RandomForestClassifier_basic']

In [299]:
rfc_loaded = joblib.load("F:\\Classification_models\\RandomForestClassifier_basic")

y_pred = rfc_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "RandomForestClassifier_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.97      0.96      0.97      1083
     dostawa       0.77      0.89      0.83      1639
       other       0.85      0.79      0.82      2411
  reklamacja       0.79      0.79      0.79       942
       zwrot       0.87      0.81      0.84      1175

    accuracy                           0.84      7250
   macro avg       0.85      0.85      0.85      7250
weighted avg       0.84      0.84      0.84      7250



In [300]:
mnb = MultinomialNB()
mnb.fit(X_train_tf, y_train)

joblib.dump(mnb, "F:\\Classification_models\\mnb_basic")

['F:\\Classification_models\\mnb_basic']

In [301]:
mnb_loaded = joblib.load("F:\\Classification_models\\mnb_basic")

y_pred = mnb_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "mnb_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.92      0.95      1083
     dostawa       0.72      0.84      0.78      1639
       other       0.70      0.75      0.72      2411
  reklamacja       0.78      0.60      0.68       942
       zwrot       0.82      0.73      0.77      1175

    accuracy                           0.77      7250
   macro avg       0.80      0.77      0.78      7250
weighted avg       0.78      0.77      0.77      7250



In [302]:
svm = SVC()
svm.fit(X_train_tf, y_train)

joblib.dump(svm, "F:\\Classification_models\\SVC_basic")

['F:\\Classification_models\\SVC_basic']

In [303]:
svm_loaded = joblib.load("F:\\Classification_models\\SVC_basic")

y_pred = svm_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "SVC_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.96      0.97      1083
     dostawa       0.79      0.89      0.84      1639
       other       0.83      0.80      0.82      2411
  reklamacja       0.79      0.80      0.79       942
       zwrot       0.88      0.81      0.84      1175

    accuracy                           0.84      7250
   macro avg       0.85      0.85      0.85      7250
weighted avg       0.85      0.84      0.84      7250



In [304]:
fig = px.scatter(df_functions_score, y="label", x="score", color="model_name", symbol="metric")
fig.show()

PCA

In [305]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [306]:
pca = PCA(n_components=2000)
X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())

file_path = 'F:\\Classification_models/PCA_X_test_train.pkl'

with open(file_path, 'wb') as f:
    pickle.dump({'X_train_pca': X_train_pca, 'X_test_pca': X_test_pca}, f)

In [307]:
file_path = 'F:\\Classification_models/PCA_X_test_train.pkl'

with open(file_path, 'rb') as f:
    data = pickle.load(f)
    X_train_pca_loaded = data['X_train_pca']
    X_test_pca_loaded = data['X_test_pca']


In [308]:
lr_PCA = LogisticRegression()
lr_PCA.fit(X_train_pca_loaded, y_train)

joblib.dump(lr_PCA, "F:\\Classification_models\\LogisticRegression_PCA")

['F:\\Classification_models\\LogisticRegression_PCA']

In [309]:
lr_PCA_loaded = joblib.load("F:\\Classification_models\\LogisticRegression_PCA")


y_pred = lr_PCA_loaded.predict(X_test_pca_loaded)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "Logistic_regression_PCA")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.96      0.97      1083
     dostawa       0.77      0.87      0.82      1639
       other       0.79      0.77      0.78      2411
  reklamacja       0.78      0.74      0.76       942
       zwrot       0.86      0.79      0.83      1175

    accuracy                           0.82      7250
   macro avg       0.83      0.83      0.83      7250
weighted avg       0.82      0.82      0.82      7250



In [310]:
rfc_PCA = RandomForestClassifier()
rfc_PCA.fit(X_train_pca_loaded, y_train)

joblib.dump(rfc_PCA, "F:\\Classification_models\\RandomForestClassifier_PCA")

['F:\\Classification_models\\RandomForestClassifier_PCA']

In [311]:
rfc_PCA_loaded = joblib.load("F:\\Classification_models\\RandomForestClassifier_PCA")

y_pred = rfc_PCA_loaded.predict(X_test_pca_loaded)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "RandomForestClassifier_PCA")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.99      0.89      0.94      1083
     dostawa       0.76      0.76      0.76      1639
       other       0.62      0.84      0.71      2411
  reklamacja       0.80      0.40      0.53       942
       zwrot       0.88      0.64      0.74      1175

    accuracy                           0.74      7250
   macro avg       0.81      0.71      0.74      7250
weighted avg       0.77      0.74      0.74      7250



In [312]:
svm_PCA = SVC()
svm_PCA.fit(X_train_pca_loaded, y_train)

joblib.dump(svm_PCA, "F:\\Classification_models\\SVC_PCA")

['F:\\Classification_models\\SVC_PCA']

In [313]:
svm_PCA_loaded = joblib.load("F:\\Classification_models\\SVC_PCA")

y_pred = svm_PCA_loaded.predict(X_test_pca_loaded)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "SVC_PCA")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.96      0.97      1083
     dostawa       0.79      0.90      0.84      1639
       other       0.84      0.80      0.82      2411
  reklamacja       0.79      0.80      0.80       942
       zwrot       0.87      0.82      0.84      1175

    accuracy                           0.85      7250
   macro avg       0.86      0.85      0.85      7250
weighted avg       0.85      0.85      0.85      7250



In [314]:
app = dash.Dash(__name__)

unique_labels = df_functions_score["model_name"].unique()
checkbox_options = [{'label': label, 'value': label} for label in unique_labels]

checkbox = dcc.Checklist(options=checkbox_options, value=[label for label in unique_labels], id='label-checkbox')

fig = px.scatter(df_functions_score, y="label", x="score", color="model_name", symbol="metric")

app.layout = html.Div([
    html.H1("Scatter Plot z checkboxem"),
    checkbox,
    dcc.Graph(figure=fig, id='scatter-plot')
], style={'backgroundColor': 'white'})

@app.callback(
    Output('scatter-plot', 'figure'),
    Input('label-checkbox', 'value')
)
def update_graph(selected_labels):
    filtered_data = df_functions_score[df_functions_score["model_name"].isin(selected_labels)]
    updated_fig = px.scatter(filtered_data, y="label", x="score", color="model_name", symbol="metric")
    return updated_fig

if __name__ == '__main__':
    app.run_server(debug=True)


Word2Vec - model

In [315]:
processed_emails = []

df_to_model['desc_transformed']

for mail in df_to_model['desc_transformed']:
    processed_emails.append(mail.split())

In [316]:
word2vec_model = Word2Vec(processed_emails, vector_size=100, window=5, min_count=1, sg=1)
word2vec_model.train(processed_emails, total_examples=len(processed_emails), epochs=100)

joblib.dump(word2vec_model, "F:\\Classification_models\\word2vec_model")

['F:\\Classification_models\\word2vec_model']

In [317]:
word2vec_model_loaded = joblib.load("F:\\Classification_models\\word2vec_model")

In [318]:
word_count = len(word2vec_model_loaded.wv)
print("Liczba załadowanych słów:", word_count)

Liczba załadowanych słów: 29877


In [319]:
type(word_count)

int

In [320]:
vectored_emails = []
vectored_emails_array = []

for email in processed_emails:
    email_vectors = [word2vec_model_loaded.wv[word] for word in email if word in word2vec_model_loaded.wv]
    sum_vector = np.sum(email_vectors, axis=0) if email_vectors else np.zeros(100)
    vectored_emails_array.append(sum_vector)
    vectored_emails.append(sum_vector)

vectored_emails_array = np.array(vectored_emails)

In [321]:
# len(vectored_emails)
len(vectored_emails_array)

36250

In [322]:
len(df_to_model['final_label_grouped'])

36250

In [323]:
X = vectored_emails_array
y = df_to_model['final_label_grouped']

In [324]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y_train)

In [325]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

joblib.dump(lr, "F:\\Classification_models\\LogisticRegression_word2vec_basic")

['F:\\Classification_models\\LogisticRegression_word2vec_basic']

In [326]:
lr_loaded = joblib.load("F:\\Classification_models\\LogisticRegression_word2vec_basic")


y_pred = lr_loaded.predict(X_test)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "LogisticRegression_word2vec_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.94      0.95      0.95      1083
     dostawa       0.71      0.84      0.77      1639
       other       0.75      0.65      0.70      2411
  reklamacja       0.71      0.67      0.69       942
       zwrot       0.75      0.78      0.77      1175

    accuracy                           0.76      7250
   macro avg       0.77      0.78      0.77      7250
weighted avg       0.76      0.76      0.76      7250



In [327]:
rfc = RandomForestClassifier()
rfc.fit(X_train_tf, y_train)

joblib.dump(rfc, "F:\\Classification_models\\RandomForestClassifier_word2vec_basic")

['F:\\Classification_models\\RandomForestClassifier_word2vec_basic']

In [328]:
rfc_loaded = joblib.load("F:\\Classification_models\\RandomForestClassifier_word2vec_basic")

y_pred = rfc_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "RandomForestClassifier_word2vec_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.97      0.97      0.97      1083
     dostawa       0.77      0.89      0.83      1639
       other       0.84      0.78      0.81      2411
  reklamacja       0.79      0.79      0.79       942
       zwrot       0.87      0.80      0.84      1175

    accuracy                           0.84      7250
   macro avg       0.85      0.85      0.85      7250
weighted avg       0.84      0.84      0.84      7250



In [329]:
mnb = MultinomialNB()
mnb.fit(X_train_tf, y_train)

joblib.dump(mnb, "F:\\Classification_models\\mnb_word2vec_basic")

['F:\\Classification_models\\mnb_word2vec_basic']

In [330]:
mnb_loaded = joblib.load("F:\\Classification_models\\mnb_word2vec_basic")

y_pred = mnb_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "mnb_word2vec_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.92      0.95      1083
     dostawa       0.72      0.84      0.78      1639
       other       0.70      0.75      0.72      2411
  reklamacja       0.78      0.60      0.68       942
       zwrot       0.82      0.73      0.77      1175

    accuracy                           0.77      7250
   macro avg       0.80      0.77      0.78      7250
weighted avg       0.78      0.77      0.77      7250



In [331]:
svm = SVC()
svm.fit(X_train_tf, y_train)

joblib.dump(svm, "F:\\Classification_models\\SVC_word2vec_basic")

['F:\\Classification_models\\SVC_word2vec_basic']

In [332]:
svm_loaded = joblib.load("F:\\Classification_models\\SVC_word2vec_basic")

y_pred = svm_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "SVC_word2vec_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.96      0.97      1083
     dostawa       0.79      0.89      0.84      1639
       other       0.83      0.80      0.82      2411
  reklamacja       0.79      0.80      0.79       942
       zwrot       0.88      0.81      0.84      1175

    accuracy                           0.84      7250
   macro avg       0.85      0.85      0.85      7250
weighted avg       0.85      0.84      0.84      7250



In [333]:
app = dash.Dash(__name__)

unique_labels = df_functions_score["model_name"].unique()
checkbox_options = [{'label': label, 'value': label} for label in unique_labels]

checkbox = dcc.Checklist(options=checkbox_options, value=[label for label in unique_labels], id='label-checkbox')

fig = px.scatter(df_functions_score, y="label", x="score", color="model_name", symbol="metric")

app.layout = html.Div([
    html.H1("Scatter Plot z checkboxem"),
    checkbox,
    dcc.Graph(figure=fig, id='scatter-plot')
], style={'backgroundColor': 'white'})

@app.callback(
    Output('scatter-plot', 'figure'),
    Input('label-checkbox', 'value')
)
def update_graph(selected_labels):
    filtered_data = df_functions_score[df_functions_score["model_name"].isin(selected_labels)]
    updated_fig = px.scatter(filtered_data, y="label", x="score", color="model_name", symbol="metric")
    return updated_fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [334]:
X = df_to_model['desc_transformed']
y = df_to_model['final_label_grouped']


In [335]:
average_length = df_to_model['desc_transformed'].apply(len).mean()

print(f"Średnia długość szeregu: {average_length}")

Średnia długość szeregu: 225.9782620689655


In [336]:
vectorizer = layers.TextVectorization(max_tokens=20000, output_sequence_length=240)
vectorizer.adapt(X)

In [337]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [338]:
word_vector_dict = {word: word2vec_model_loaded.wv[word] for word in word2vec_model_loaded.wv.index_to_key}

In [339]:
words = list(word_vector_dict.keys())

num_tokens = len(words) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = word_vector_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


Converted 18546 words (1454 misses)


In [340]:
label_encoder = LabelEncoder()

encoded_labels = label_encoder.fit_transform(y)
y_train_onehot = to_categorical(encoded_labels)

In [341]:

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y_train_onehot,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y_train_onehot)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y_train)

In [342]:
X_train = vectorizer(np.array([[s] for s in X_train])).numpy()
X_val = vectorizer(np.array([[s] for s in X_val])).numpy()

y_train = np.array(y_train)
y_val = np.array(y_val)

In [343]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    trainable=False,
)
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

In [344]:
int_sequences_input = keras.Input(shape=(None,), dtype="int32")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(5, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         2987900   
                                                                 
 conv1d_6 (Conv1D)           (None, None, 128)         64128     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, None, 128)        0         
 1D)                                                       

In [345]:
model.compile(
    loss="categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(np.array(X_train), np.array(y_train), batch_size=128, epochs=20, validation_data=(np.array(X_val), np.array(y_val)))

model.save("F:\\Classification_models\\neural_network_on_word2vec")

Epoch 1/20
182/182 [==============================] - 32s 168ms/step - loss: 0.8302 - acc: 0.6680 - val_loss: 0.5927 - val_acc: 0.7686
Epoch 2/20
182/182 [==============================] - 30s 168ms/step - loss: 0.5506 - acc: 0.7912 - val_loss: 0.6214 - val_acc: 0.7791
Epoch 3/20
182/182 [==============================] - 31s 169ms/step - loss: 0.4680 - acc: 0.8220 - val_loss: 0.6417 - val_acc: 0.7552
Epoch 4/20
182/182 [==============================] - 31s 170ms/step - loss: 0.4092 - acc: 0.8446 - val_loss: 0.4855 - val_acc: 0.8138
Epoch 5/20
182/182 [==============================] - 31s 171ms/step - loss: 0.3525 - acc: 0.8655 - val_loss: 0.5391 - val_acc: 0.8210
Epoch 6/20
182/182 [==============================] - 31s 169ms/step - loss: 0.2985 - acc: 0.8852 - val_loss: 0.5859 - val_acc: 0.8086
Epoch 7/20
182/182 [==============================] - 31s 170ms/step - loss: 0.2513 - acc: 0.9043 - val_loss: 0.5299 - val_acc: 0.8200
Epoch 8/20
182/182 [==============================] - 3

INFO:tensorflow:Assets written to: F:\Classification_models\neural_network_on_word2vec\assets


INFO:tensorflow:Assets written to: F:\Classification_models\neural_network_on_word2vec\assets


In [346]:
from tensorflow.keras.models import load_model

neural_network_w2v = load_model("F:\\Classification_models\\neural_network_on_word2vec")

In [347]:
class_names = ['anulacja', 'dostawa', 'other', 'reklamacja', 'zwrot']

string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = neural_network_w2v(x)
end_to_end_model = keras.Model(string_input, preds)

prediction_neural_network = []

for sample in X_test:
    input_data = [[sample]]
    input_tensor = tf.convert_to_tensor(input_data, dtype=tf.string)
    probabilities = end_to_end_model(input_tensor)
    predicted_class_index = np.argmax(probabilities[0])
    prediction_neural_network.append(predicted_class_index)


In [348]:
df_functions_score_temp = fl.model_score_to_df(np.argmax(y_test, axis=1), prediction_neural_network, "neural_network_word2vec")
df_functions_score_temp['label'] = df_functions_score_temp['label'].map({i: class_names[i] for i in range(len(class_names))})
df_functions_score_temp['label'][15] = "overall"
df_functions_score = pd.concat([df_functions_score, df_functions_score_temp], ignore_index=True)



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




In [349]:
df_functions_score_temp

,model_name,label,score,metric
0,neural_network_word2vec,anulacja,0.980989,precision
0,neural_network_word2vec,anulacja,0.952909,recall
0,neural_network_word2vec,anulacja,0.966745,f1-score
0,neural_network_word2vec,dostawa,0.757545,precision
0,neural_network_word2vec,dostawa,0.903600,recall
0,neural_network_word2vec,dostawa,0.824151,f1-score
0,neural_network_word2vec,other,0.856801,precision
0,neural_network_word2vec,other,0.742016,recall
0,neural_network_word2vec,other,0.795288,f1-score
0,neural_network_word2vec,reklamacja,0.744041,precision


In [350]:
app = dash.Dash(__name__)

unique_labels = df_functions_score["model_name"].unique()
checkbox_options = [{'label': label, 'value': label} for label in unique_labels]

checkbox = dcc.Checklist(options=checkbox_options, value=[label for label in unique_labels], id='label-checkbox')

fig = px.scatter(df_functions_score, y="label", x="score", color="model_name", symbol="metric")

app.layout = html.Div([
    html.H1("Scatter Plot z checkboxem"),
    checkbox,
    dcc.Graph(figure=fig, id='scatter-plot')
], style={'backgroundColor': 'white'})

@app.callback(
    Output('scatter-plot', 'figure'),
    Input('label-checkbox', 'value')
)
def update_graph(selected_labels):
    filtered_data = df_functions_score[df_functions_score["model_name"].isin(selected_labels)]
    updated_fig = px.scatter(filtered_data, y="label", x="score", color="model_name", symbol="metric")
    return updated_fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [351]:
nlp = spacy.blank("pl")
optimizer=nlp.initialize()

In [352]:
df_spacy = pd.DataFrame(df_to_model[['desc_transformed', 'final_label_grouped']])
df_spacy.reset_index(inplace=True)

In [353]:
class_names = ['anulacja', 'dostawa', 'other', 'reklamacja', 'zwrot']

categories = df_spacy['final_label_grouped'].values.reshape(-1, 1)

encoder = OneHotEncoder()
one_hot_encoded = encoder.fit_transform(categories)
one_hot_array = one_hot_encoded.toarray()

df_encoded = pd.DataFrame(one_hot_array, columns=class_names).astype(int)

label = list(df_encoded.columns)
df_encoded = df_encoded.to_dict('index')

In [354]:
dataset = list(zip(df_to_model['desc_transformed'], [{'categories':cat} for cat in df_encoded.values()]))
print(dataset[0])

('klient pytanie produkt ASIN   połączenie dachówka wentylacyjny   link zapytanie web_site pozdrawiać persName', {'categories': {'anulacja': 0, 'dostawa': 0, 'other': 1, 'reklamacja': 0, 'zwrot': 0}})


In [355]:
train_data, test_data = train_test_split(dataset,
                                        test_size=0.25,
                                        random_state=42)
dev_data, test_data = train_test_split(test_data, 
                                       test_size=0.4,
                                       random_state=42)

print(f"Total: {len(dataset)} - Train:  {len(train_data)} - Dev: {len(dev_data)} - Test: {len(test_data)}")

Total: 36250 - Train:  27187 - Dev: 5437 - Test: 3626


In [356]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()

    for text, labels in data:
        doc = spacy.tokens.Doc(nlp.vocab, words=text.split())
        doc.cats.update(labels['categories'])
        db.add(doc)

    db.to_disk(outfile)

convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

In [357]:
textcat = nlp.add_pipe("textcat_multilabel")
for i in label:
    textcat.add_label(i)

In [358]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [359]:
! python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: []
ℹ Initial learn rate: 0.001
E    #       SCORE 
---  ------  ------
  0       0    0.00
  0     200    0.00
  0     400    0.00
  0     600    0.00
  0     800    0.00
  0    1000    0.00
  0    1200    0.00
  0    1400    0.00
  0    1600    0.00
✔ Saved pipeline to output directory
output\model-last


In [360]:
output_dir = 'F:\\Classification_models\\spaCy_model'
nlp.to_disk(output_dir)

In [361]:
saved_model_dir = 'F:\Classification_models\\spaCy_model'
nlp = spacy.load(saved_model_dir)

In [362]:
! python -m spacy evaluate ./model/model-best/ ./test.spacy

ℹ Using CPU

================================== Results ==================================

TOK                   -     
TEXTCAT (macro AUC)   95.58 
SPEED                 218748


=========================== Textcat F (per label) ===========================

                 P       R       F
anulacja     98.14   95.32   96.71
dostawa      76.79   85.20   80.78
other        86.32   69.25   76.85
reklamacja   79.26   66.08   72.07
zwrot        85.58   79.20   82.27


======================== Textcat ROC AUC (per label) ========================

             ROC AUC
anulacja        0.99
dostawa         0.95
other           0.92
reklamacja      0.96
zwrot           0.96



c:\Users\48575\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.6.1 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [364]:
! python -m spacy validate


⠙ Loading compatibility table...
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.2) =================
ℹ spaCy installation:
c:\Users\48575\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy

NAME              SPACY            VERSION                            
en_core_web_sm    >=3.7.2,<3.8.0   3.7.1   ✔
pl_core_news_lg   >=3.7.0,<3.8.0   3.7.0   ✔
pl_core_news_md   >=3.7.0,<3.8.0   3.7.0   ✔
pl_core_news_sm   >=3.7.0,<3.8.0   3.7.0   ✔



In [363]:
# Function to predict using SpaCy model
def predict_spaCy(text):
    doc = nlp(text)
    

    label = max(doc.cats, key=doc.cats.get)
    
    return label

text_to_classify = "klient pytanie produkt ASIN połączenie dachówka wentylacyjny link zapytanie web_site pozdrawiać persName"
predicted_label = predict_spaCy(text_to_classify)

print("Predicted Label:", predicted_label)

ValueError: Cannot get dimension 'nO' for model 'sparse_linear': value unset

In [ ]:
def predict_spaCy (text):
    doc = nlp(text)
    label = max(doc.cats, key=doc.cats.get)
    return label

In [ ]:
y_pred= []
for recod in dev_data:
    y_pred.append(recod[0])

In [ ]:
predict_spaCy("dzien dobry klient otrzymać paczka termin doręczenie minąć number number ostatni status śledzić number number prosić weryfikacja pozdrawiać persName")

ValueError: Cannot get dimension 'nO' for model 'sparse_linear': value unset

In [ ]:
for x in y_pred:
    print(predict_spaCy(x))

ValueError: Cannot get dimension 'nO' for model 'sparse_linear': value unset

In [ ]:
y_pred

['dzien dobry klient otrzymać paczka termin doręczenie minąć number number ostatni status śledzić number number prosić weryfikacja pozdrawiać persName',
 'zamówienie anulować prośba klient persName',
 'klient prosić anulować \xa0  \xa0 items_would_arrive_too_late \xa0  persName',
 'dzień dobry kupujący prosić faktura zamówienie pozdrawiać persName',
 'dzień dobry klient prosić anulacja \xa0  pozdrawiać persName \xa0',
 'dzień dobry śledzeć informacja wysłaniu paczka termin dostawa minąć number number prosić weryfikacja klient zapytać spodziewać paczka pozdrawiać persName',
 'dzień dobry \xa0  kupować prosić anulować zamówienie \xa0  wyraz szacunek \xa0  persName \xa0',
 'dzień dobry kupujący otrzymać uszkodzić towar zdjęcie załącznik nowy sam kolor biały pozdrawiać persName',
 'siłownik dopiero prawidłowo pracować obciążyć materace zmontować cały łóżko przykręcić koniec siłownik położyć materac zamknąć',
 'dzień dobry kupować zwrócić towar prosić etykieta pozdrawiać persName',
 'dzień 

In [ ]:
dev_data[1][0]

'zamówienie anulować prośba klient persName'

In [ ]:
y_test_pred = dev_data

In [ ]:
i == 1

False

Optimization of Logistic Regression, Random Forest Classifier, and Naive Bayes

In [ ]:
X = df['Description_cleaned']
y = df['final_label']

NameError: name 'df' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [ ]:
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [ ]:
X_train_tf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:


lr = LogisticRegression()
lr.fit(X_train_tf, y_train)

y_pred = lr.predict(X_test_tf)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.82      0.74      0.78       200
    anulacja       0.97      0.95      0.96      1135
        brak       0.77      0.48      0.59       164
     dostawa       0.72      0.91      0.80      1647
     faktura       1.00      0.97      0.98       379
        inne       0.85      0.52      0.64        33
niekompletny       0.89      0.46      0.60        90
  opóźnienie       0.89      0.59      0.71        83
      paczka       0.90      0.58      0.70       260
     ponowny       0.85      0.35      0.50        31
   przesyłka       0.86      0.50      0.63        72
     pytanie       0.88      0.91      0.89       526
      refund       0.93      0.65      0.77       153
  reklamacja       0.67      0.87      0.76       867
  uszkodzony       0.82      0.51      0.63       175
     wysyłka       0.75      0.14      0.23       261
       zwrot       0.85      0.83      0.84      1171

    accuracy              

In [ ]:
param_grid = {
    'penalty': ['l1','l2','None'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'fit_intercept': [True, False],
    'tol' : [0.001,0.01],
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter': [100, 200],
    'multi_class': ['auto', 'multinomial'],
}

In [ ]:
# searcher=RandomizedSearchCV(lr, param_grid)
# searcher.fit(X_train_tf, y_train)

In [ ]:
# joblib.dump(searcher, "LogisticRegression_randomizedSearchCV")

In [ ]:
searcher = joblib.load("LogisticRegression_randomizedSearchCV")

In [ ]:
print(searcher.best_score_)
print(searcher.best_params_)

0.7918849153934211
{'solver': 'saga', 'penalty': 'l2', 'multi_class': 'ovr', 'fit_intercept': True, 'C': 10}


In [ ]:
lr = LogisticRegression(tol=0.01, solver='newton-cg', penalty='l2', multi_class='multinomial', 
                        max_iter=200, fit_intercept=True, C=10
                        )
lr.fit(X_train_tf, y_train)

y_pred = lr.predict(X_test_tf)


print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.80      0.78      0.79       200
    anulacja       0.97      0.96      0.97      1135
        brak       0.76      0.66      0.71       164
     dostawa       0.75      0.90      0.82      1647
     faktura       0.99      0.98      0.99       379
        inne       0.86      0.55      0.67        33
niekompletny       0.84      0.59      0.69        90
  opóźnienie       0.88      0.69      0.77        83
      paczka       0.86      0.66      0.75       260
     ponowny       0.65      0.48      0.56        31
   przesyłka       0.75      0.67      0.71        72
     pytanie       0.89      0.91      0.90       526
      refund       0.87      0.69      0.77       153
  reklamacja       0.72      0.86      0.79       867
  uszkodzony       0.76      0.61      0.68       175
     wysyłka       0.70      0.17      0.28       261
       zwrot       0.87      0.83      0.85      1171

    accuracy              

In [ ]:

rfc = RandomForestClassifier()
rfc.fit(X_train_tf, y_train)

y_pred = rfc.predict(X_test_tf)



print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.83      0.69      0.75       200
    anulacja       0.97      0.95      0.96      1135
        brak       0.83      0.51      0.63       164
     dostawa       0.71      0.93      0.80      1647
     faktura       0.99      0.97      0.98       379
        inne       0.86      0.55      0.67        33
niekompletny       0.85      0.37      0.51        90
  opóźnienie       0.85      0.57      0.68        83
      paczka       0.90      0.55      0.69       260
     ponowny       0.80      0.39      0.52        31
   przesyłka       0.80      0.56      0.66        72
     pytanie       0.90      0.90      0.90       526
      refund       0.91      0.67      0.77       153
  reklamacja       0.69      0.86      0.77       867
  uszkodzony       0.84      0.45      0.59       175
     wysyłka       0.84      0.15      0.25       261
       zwrot       0.84      0.83      0.84      1171

    accuracy              

In [ ]:
i == 1

In [ ]:
param_grid = {
    'n_estimators': [50, 125, 200],  # Range of values for n_estimators
    'max_depth': [None, 5, 13, 20],  # Including None for no max depth
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'criterion': ['gini', 'entropy']
}

In [ ]:
# searcher=RandomizedSearchCV(rfc, param_grid)
# searcher.fit(X_train_tf, y_train)

In [ ]:
# joblib.dump(searcher, "RandomForestClassifier_randomizedSearchCV")

In [ ]:
searcher = joblib.load("LogisticRegression_randomizedSearchCV")

In [ ]:
print(searcher.best_score_)
print(searcher.best_params_)

0.7918849153934211
{'solver': 'saga', 'penalty': 'l2', 'multi_class': 'ovr', 'fit_intercept': True, 'C': 10}


In [ ]:
# rfc = RandomForestClassifier(n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=None, max_depth=None, criterion='gini')
# rfc.fit(X_train_tf, y_train)

# y_pred = rfc.predict(X_test_tf)

# from sklearn.metrics import classification_report

# print(classification_report(y_test, y_pred))